In [1]:
# Cell 1 — configuration, imports & maps

# 1. Your current WebSocket session ID from the browser
SESSION_ID = "t9ITJuz6CopZzndZAC_O"

import json
import socketio
from threading import Thread

# 2. Load your JSON definitions
with open("npcs_minified.json", encoding="utf-8") as f:
    NPC_MAP = {npc["_id"]: npc["name"] for npc in json.load(f)}

with open("items.json", encoding="utf-8") as f:
    ITEM_MAP = {item["id"]: item["name"] for item in json.load(f)}

# 3. Opcode constants
OP_HIT_EVENT     = 8    # [attackerId, defenderId, damage]
OP_SKILL_XP      = 7    # [skillId, xpGained]
OP_INVENTORY_UPD = 6    # [itemId, newCount]
OP_HP_EVENT      = 13   # [entityId, hpOrXp]
OP_DROP_EVENT    = 5    # [msgId, itemId, qty, ..., x, y]

# 4. Known NPC IDs (from your JSON)
BANDIT_ID  = 9
PLAYER_ID  = None  # will auto-detect on first hit

# 5. Initialize the Socket.IO client
sio = socketio.Client(logger=False, reconnection=True)


In [2]:
# Cell 2 — define your callbacks

@sio.event
def connect():
    # This prints the new sid (should match your SESSION_ID if attached correctly)
    print("✅ Connected!  sid=", sio.sid)

@sio.on("0")
def on_batch(data):
    global PLAYER_ID

    for op, payload in data:

        # ── Combat hit events ──
        if op == OP_HIT_EVENT:
            attacker, defender, dmg = payload

            # auto-detect your player’s entity ID
            if attacker not in (BANDIT_ID,) and PLAYER_ID is None:
                PLAYER_ID = attacker
                print(f"🔍 Detected PLAYER_ID = {PLAYER_ID}")

            name_a = NPC_MAP.get(attacker, "Player")
            name_d = NPC_MAP.get(defender,  "Player")
            print(f"{name_a} hit {name_d}: {dmg}")

        # ── Skill XP gained events ──
        elif op == OP_SKILL_XP:
            skill_id, xp = payload
            print(f"⭐ Skill {skill_id} XP gained: {xp}")

        # ── Inventory updates (coins etc.) ──
        elif op == OP_INVENTORY_UPD:
            item_id, new_count = payload
            item = ITEM_MAP.get(item_id, f"Item#{item_id}")
            print(f"📦 Inventory: {item} now {new_count}")

        # ── HP / death events ──
        elif op == OP_HP_EVENT:
            entity_id, hp_val = payload
            who = NPC_MAP.get(entity_id, "Player")
            print(f"❤️ {who} HP/XP event: {hp_val}")

        # ── Item drops ──
        elif op == OP_DROP_EVENT:
            _, item_id, qty, *_ = payload
            item = ITEM_MAP.get(item_id, f"Item#{item_id}")
            print(f"→ Drop: {item} x{qty}")

        # ── (optional) handle other opcodes here ──


In [3]:
# Cell 3 — start a raw WebSocket listener with file logging

import websocket
import threading
import logging

# 1. Configure logging to append to combat.log
logging.basicConfig(
    filename="combat.log",
    filemode="a",
    format="%(asctime)s %(message)s",
    datefmt="%H:%M:%S",
    level=logging.INFO
)
logger = logging.getLogger("hs_tailer")

# Build the exact URL that your browser uses (with your sid):
WS_URL = (
    "wss://server1.highspell.com:8888"
    "/socket.io/?EIO=4&transport=websocket"
    f"&sid={SESSION_ID}"
)

def on_message(ws, message):
    if not message.startswith("42"):
        return
    payload = json.loads(message[2:])
    _, batch = payload

    for op, data in batch:
        if op == OP_HIT_EVENT:
            attacker, defender, dmg = data
            global PLAYER_ID
            if attacker not in (BANDIT_ID,) and PLAYER_ID is None:
                PLAYER_ID = attacker
                logger.info(f"🔍 Detected PLAYER_ID = {attacker}")
            name_a = NPC_MAP.get(attacker, "Player")
            name_d = NPC_MAP.get(defender,  "Player")
            line = f"{name_a} hit {name_d}: {dmg}"
            print(line)
            logger.info(line)

        elif op == OP_DROP_EVENT:
            _, item_id, qty, *_ = data
            item = ITEM_MAP.get(item_id, f"Item#{item_id}")
            line = f"→ Drop: {item} x{qty}"
            print(line)
            logger.info(line)

        # … mirror the same print+logger.info for other opcodes …

def on_error(ws, error):
    print("⚠️ WebSocket error:", error)
    logger.error(f"WebSocket error: {error}")

def on_close(ws, code, reason):
    line = f"❌ Closed ({code}): {reason}"
    print(line)
    logger.warning(line)

def start_ws():
    ws = websocket.WebSocketApp(
        WS_URL,
        on_message=on_message,
        on_error=on_error,
        on_close=on_close,
    )
    ws.run_forever()

threading.Thread(target=start_ws, daemon=True).start()
print("Raw WebSocket client running—watch combat.log for live events.")


Raw WebSocket client running—watch combat.log for live events.


❌ Closed (None): None
